# 04 - Embeddings: Token et Positional

## Objectifs Pédagogiques

Dans ce notebook, nous allons:
1. **Comprendre les embeddings de tokens** - Comment convertir des IDs discrets en vecteurs denses
2. **Implémenter l'encodage positionnel** - Comment encoder la position des tokens dans une séquence
3. **Combiner les deux types d'embeddings** - Comment créer les embeddings d'entrée finaux
4. **Visualiser les patterns** - Observer les structures dans l'espace d'embedding

## Pourquoi les Embeddings?

Les transformers ne peuvent pas travailler directement avec des tokens discrets (mots, sous-mots). Ils ont besoin de représentations continues (vecteurs) qui capturent:
- **Le contenu sémantique** (via les token embeddings)
- **La position dans la séquence** (via les positional embeddings)

## Structure du Notebook

**Partie 1**: Token Embeddings (From Scratch + PyTorch)
**Partie 2**: Positional Embeddings (From Scratch + PyTorch)
**Partie 3**: Combinaison et Visualisation

In [ ]:
# Imports nécessaires
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sys
import os

# Ajouter le chemin vers les modules src
sys.path.append('../../src')

# Import des implémentations
from embeddings.token_embedding import (
    create_token_embedding_matrix,
    token_embedding_lookup,
    TokenEmbedding,
    validate_embedding_dimensions,
    check_embedding_shape
)
from embeddings.positional_embedding import (
    create_sinusoidal_positional_encoding,
    get_positional_encoding,
    PositionalEmbedding,
    create_position_ids,
    combine_token_and_positional_embeddings,
    validate_positional_encoding_dimensions
)

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports réussis!")

---
# Partie 1: Token Embeddings

## Concept

Un **token embedding** convertit un ID de token (entier) en un vecteur dense de dimension `d_model`.

### Formule Mathématique

Soit $E \in \mathbb{R}^{\text{vocab\_size} \times d_{\text{model}}}$ la matrice d'embedding.

Pour un token avec ID $i$, l'embedding est simplement la ligne $i$ de la matrice:

$$\text{embedding}(i) = E[i] \in \mathbb{R}^{d_{\text{model}}}$$

Pour une séquence de tokens $[i_1, i_2, ..., i_n]$, on obtient:

$$\text{embeddings} = [E[i_1], E[i_2], ..., E[i_n]] \in \mathbb{R}^{n \times d_{\text{model}}}$$

### Dimensions

- **Input**: Token IDs de shape `(batch_size, seq_len)`
- **Matrice d'embedding**: Shape `(vocab_size, d_model)`
- **Output**: Embeddings de shape `(batch_size, seq_len, d_model)`

In [ ]:
# ============================================
# IMPLÉMENTATION 1: From Scratch (NumPy)
# ============================================

print("=" * 60)
print("TOKEN EMBEDDINGS - FROM SCRATCH (NumPy)")
print("=" * 60)

# Configuration
vocab_size = 1000  # Vocabulaire de 1000 tokens
d_model = 128      # Vecteurs de dimension 128
batch_size = 4     # Batch de 4 séquences
seq_len = 10       # Séquences de longueur 10

print(f"\nConfiguration:")
print(f"  - Vocabulaire: {vocab_size} tokens")
print(f"  - Dimension d'embedding: {d_model}")
print(f"  - Taille du batch: {batch_size}")
print(f"  - Longueur de séquence: {seq_len}")

# Étape 1: Créer la matrice d'embedding
print(f"\n1. Création de la matrice d'embedding:")
embedding_matrix = create_token_embedding_matrix(vocab_size, d_model)
print(f"   ✓ Shape: {embedding_matrix.shape}")
print(f"   ✓ Nombre de paramètres: {vocab_size * d_model:,}")

# Étape 2: Créer des token IDs aléatoires
print(f"\n2. Création de token IDs:")
token_ids = np.random.randint(0, vocab_size, size=(batch_size, seq_len))
print(f"   ✓ Shape: {token_ids.shape}")
print(f"   ✓ Exemple (première séquence): {token_ids[0]}")

# Étape 3: Lookup des embeddings
print(f"\n3. Lookup des embeddings:")
embeddings = token_embedding_lookup(token_ids, embedding_matrix)
print(f"   ✓ Shape: {embeddings.shape}")
print(f"   ✓ Vérification: {embeddings.shape} == ({batch_size}, {seq_len}, {d_model})")

# Étape 4: Inspection d'un embedding
print(f"\n4. Inspection d'un embedding:")
token_id = token_ids[0, 0]
embedding_vector = embeddings[0, 0]
print(f"   Token ID: {token_id}")
print(f"   Embedding (5 premiers éléments): {embedding_vector[:5]}")
print(f"   Norme L2: {np.linalg.norm(embedding_vector):.4f}")

print("\n" + "=" * 60)
print("✓ Implémentation NumPy terminée!")
print("=" * 60)

## Implémentation PyTorch: nn.Embedding

PyTorch fournit `nn.Embedding`, une couche optimisée qui:
- Stocke la matrice d'embedding comme paramètres apprenables
- Effectue des lookups efficaces sur GPU
- Supporte la différentiation automatique

### Méthodes PyTorch Utilisées

- **`nn.Embedding(num_embeddings, embedding_dim)`**: Crée une couche d'embedding
  - `num_embeddings`: Taille du vocabulaire
  - `embedding_dim`: Dimension des vecteurs
  - Attribut `weight`: Matrice d'embedding de shape `(vocab_size, d_model)`

- **Forward pass**: Input `(batch_size, seq_len)` → Output `(batch_size, seq_len, d_model)`

In [ ]:
# ============================================
# IMPLÉMENTATION 2: PyTorch (nn.Module)
# ============================================

print("=" * 60)
print("TOKEN EMBEDDINGS - PYTORCH (nn.Embedding)")
print("=" * 60)

# Configuration (même que NumPy pour comparaison)
vocab_size = 1000
d_model = 128
batch_size = 4
seq_len = 10

print(f"\nConfiguration:")
print(f"  - Vocabulaire: {vocab_size} tokens")
print(f"  - Dimension d'embedding: {d_model}")

# Étape 1: Créer la couche TokenEmbedding
print(f"\n1. Création de la couche TokenEmbedding:")
token_emb = TokenEmbedding(vocab_size=vocab_size, d_model=d_model)
print(f"   ✓ Couche créée: {token_emb}")

# Compter les paramètres
num_params = sum(p.numel() for p in token_emb.parameters())
print(f"   ✓ Nombre de paramètres: {num_params:,}")

# Étape 2: Créer des token IDs
print(f"\n2. Création de token IDs:")
token_ids_torch = torch.randint(0, vocab_size, (batch_size, seq_len))
print(f"   ✓ Shape: {token_ids_torch.shape}")
print(f"   ✓ Type: {token_ids_torch.dtype}")
print(f"   ✓ Exemple: {token_ids_torch[0]}")

# Étape 3: Forward pass
print(f"\n3. Forward pass:")
embeddings_torch = token_emb(token_ids_torch)
print(f"   ✓ Shape: {embeddings_torch.shape}")
print(f"   ✓ Type: {embeddings_torch.dtype}")
print(f"   ✓ Device: {embeddings_torch.device}")

# Étape 4: Vérification des dimensions
print(f"\n4. Vérification des dimensions:")
check_embedding_shape(token_ids_torch, embeddings_torch, d_model)

# Étape 5: Statistiques
print(f"5. Statistiques des embeddings:")
print(f"   - Moyenne: {embeddings_torch.mean().item():.6f}")
print(f"   - Écart-type: {embeddings_torch.std().item():.6f}")
print(f"   - Min: {embeddings_torch.min().item():.6f}")
print(f"   - Max: {embeddings_torch.max().item():.6f}")

print("\n" + "=" * 60)
print("✓ Implémentation PyTorch terminée!")
print("=" * 60)

---
# Partie 2: Positional Embeddings

## Problème

Le mécanisme d'attention est **position-agnostic**: il traite tous les tokens de la même manière, peu importe leur position. Pour que le modèle comprenne l'ordre des mots, nous devons ajouter des **encodages positionnels**.

## Solution: Encodage Sinusoïdal

Vaswani et al. (2017) ont proposé d'utiliser des fonctions sinusoïdales:

### Formules Mathématiques

Pour une position $\text{pos}$ et une dimension $i$:

$$PE_{(\text{pos}, 2i)} = \sin\left(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\right)$$

$$PE_{(\text{pos}, 2i+1)} = \cos\left(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\right)$$

où:
- $\text{pos}$ est la position dans la séquence (0, 1, 2, ...)
- $i$ est l'indice de la dimension (0, 1, 2, ..., $d_{\text{model}}/2 - 1$)
- Les dimensions **paires** (0, 2, 4, ...) utilisent $\sin$
- Les dimensions **impaires** (1, 3, 5, ...) utilisent $\cos$

### Intuition

- **Fréquences variées**: Les dimensions basses ont des fréquences élevées (changent rapidement), les dimensions hautes ont des fréquences basses (changent lentement)
- **Positions relatives**: Le modèle peut apprendre à détecter les positions relatives grâce aux patterns sinusoïdaux
- **Extrapolation**: Peut généraliser à des séquences plus longues que celles vues pendant l'entraînement

### Dimensions

- **Matrice d'encodage**: Shape `(max_seq_len, d_model)`
- **Input**: Positions de shape `(batch_size, seq_len)`
- **Output**: Encodages de shape `(batch_size, seq_len, d_model)`

In [ ]:
# ============================================
# IMPLÉMENTATION 1: From Scratch (NumPy)
# ============================================

print("=" * 60)
print("POSITIONAL ENCODINGS - FROM SCRATCH (NumPy)")
print("=" * 60)

# Configuration
max_seq_len = 100  # Séquences jusqu'à 100 tokens
d_model = 128      # Dimension (doit être pair)
batch_size = 4
seq_len = 10

print(f"\nConfiguration:")
print(f"  - Longueur max: {max_seq_len}")
print(f"  - Dimension: {d_model}")
print(f"  - Batch: {batch_size}")
print(f"  - Séquence: {seq_len}")

# Étape 1: Créer la matrice d'encodage positionnel
print(f"\n1. Création de la matrice d'encodage positionnel:")
pe_matrix = create_sinusoidal_positional_encoding(max_seq_len, d_model)
print(f"   ✓ Shape: {pe_matrix.shape}")
print(f"   ✓ Type: Sinusoïdal (sin/cos)")

# Étape 2: Créer des IDs de positions
print(f"\n2. Création des IDs de positions:")
position_ids = create_position_ids(batch_size, seq_len)
print(f"   ✓ Shape: {position_ids.shape}")
print(f"   ✓ Exemple: {position_ids[0]}")

# Étape 3: Récupérer les encodages
print(f"\n3. Récupération des encodages:")
pos_encodings = get_positional_encoding(position_ids, pe_matrix)
print(f"   ✓ Shape: {pos_encodings.shape}")
print(f"   ✓ Vérification: {pos_encodings.shape} == ({batch_size}, {seq_len}, {d_model})")

# Étape 4: Propriétés sinusoïdales
print(f"\n4. Propriétés sinusoïdales:")
print(f"   - Valeurs dans [-1, 1]: {np.all(np.abs(pe_matrix) <= 1.0)}")
print(f"   - Moyenne: {np.mean(pe_matrix):.6f}")
print(f"   - Écart-type: {np.std(pe_matrix):.6f}")

# Étape 5: Inspection d'un encodage
print(f"\n5. Inspection d'un encodage (position 0):")
pos_0 = pe_matrix[0]
print(f"   Dimensions paires (sin): {pos_0[0::2][:5]}")
print(f"   Dimensions impaires (cos): {pos_0[1::2][:5]}")

print("\n" + "=" * 60)
print("✓ Implémentation NumPy terminée!")
print("=" * 60)

## Visualisation: Heatmap des Encodages Positionnels

Visualisons les patterns sinusoïdaux dans l'encodage positionnel.

In [ ]:
# Visualisation de la matrice d'encodage positionnel
fig, ax = plt.subplots(figsize=(12, 8))

# Afficher les 50 premières positions et toutes les dimensions
num_positions_to_show = 50
pe_to_plot = pe_matrix[:num_positions_to_show, :]

# Créer le heatmap
im = ax.imshow(pe_to_plot, cmap='RdBu', aspect='auto', vmin=-1, vmax=1)

# Labels et titre
ax.set_xlabel('Dimension', fontsize=12)
ax.set_ylabel('Position', fontsize=12)
ax.set_title('Encodage Positionnel Sinusoïdal\n(Heatmap des 50 premières positions)', 
             fontsize=14, fontweight='bold')

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Valeur', rotation=270, labelpad=20)

# Annotations
ax.text(0.02, 0.98, 'Dimensions paires: sin\nDimensions impaires: cos',
        transform=ax.transAxes, fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  - Les dimensions basses (gauche) changent rapidement avec la position")
print("  - Les dimensions hautes (droite) changent lentement")
print("  - Patterns alternés sin/cos créent des structures uniques pour chaque position")

## Implémentation PyTorch: PositionalEmbedding

PyTorch permet de stocker les encodages positionnels comme un **buffer** (non-entraînable).

### Méthodes PyTorch Utilisées

- **`register_buffer()`**: Enregistre un tensor comme partie du module mais non-entraînable
  - Le buffer est sauvegardé/chargé avec le modèle
  - Déplacé automatiquement sur GPU avec le modèle
  - Pas de gradients calculés

- **`torch.arange()`**: Crée une séquence [0, 1, 2, ..., n-1]
- **`torch.sin()` / `torch.cos()`**: Fonctions trigonométriques

### Différence Clé

- **TokenEmbedding**: Poids **apprenables** (mis à jour pendant l'entraînement)
- **PositionalEmbedding**: Poids **fixes** (calculés une fois, jamais modifiés)

In [ ]:
# ============================================
# IMPLÉMENTATION 2: PyTorch (nn.Module)
# ============================================

print("=" * 60)
print("POSITIONAL ENCODINGS - PYTORCH")
print("=" * 60)

# Configuration
max_seq_len = 100
d_model = 128
batch_size = 4
seq_len = 10

print(f"\nConfiguration:")
print(f"  - Longueur max: {max_seq_len}")
print(f"  - Dimension: {d_model}")

# Étape 1: Créer la couche PositionalEmbedding
print(f"\n1. Création de la couche PositionalEmbedding:")
pos_emb = PositionalEmbedding(max_seq_len=max_seq_len, d_model=d_model)
print(f"   ✓ Couche créée: {pos_emb}")

# Vérifier que les poids ne sont pas entraînables
num_params = sum(p.numel() for p in pos_emb.parameters())
print(f"   ✓ Paramètres entraînables: {num_params}")
print(f"   ✓ Note: Les encodages sont FIXES (non-entraînables)")

# Étape 2: Créer des token IDs (les positions sont créées automatiquement)
print(f"\n2. Création de token IDs:")
token_ids_torch = torch.randint(0, 1000, (batch_size, seq_len))
print(f"   ✓ Shape: {token_ids_torch.shape}")

# Étape 3: Forward pass
print(f"\n3. Forward pass:")
pos_encodings_torch = pos_emb(token_ids_torch)
print(f"   ✓ Shape: {pos_encodings_torch.shape}")
print(f"   ✓ Type: {pos_encodings_torch.dtype}")
print(f"   ✓ Device: {pos_encodings_torch.device}")

# Étape 4: Statistiques
print(f"\n4. Statistiques:")
print(f"   - Moyenne: {pos_encodings_torch.mean().item():.6f}")
print(f"   - Écart-type: {pos_encodings_torch.std().item():.6f}")
print(f"   - Min: {pos_encodings_torch.min().item():.6f}")
print(f"   - Max: {pos_encodings_torch.max().item():.6f}")
print(f"   - Dans [-1, 1]: {torch.all(torch.abs(pos_encodings_torch) <= 1.0).item()}")

print("\n" + "=" * 60)
print("✓ Implémentation PyTorch terminée!")
print("=" * 60)

---
# Partie 3: Combinaison des Embeddings

## Formule Finale

Les embeddings d'entrée du transformer sont la **somme** des token embeddings et des positional embeddings:

$$E_{\text{input}} = E_{\text{token}} + E_{\text{position}}$$

où:
- $E_{\text{token}} \in \mathbb{R}^{\text{batch\_size} \times \text{seq\_len} \times d_{\text{model}}}$ encode le **contenu sémantique**
- $E_{\text{position}} \in \mathbb{R}^{\text{batch\_size} \times \text{seq\_len} \times d_{\text{model}}}$ encode la **position**
- $E_{\text{input}} \in \mathbb{R}^{\text{batch\_size} \times \text{seq\_len} \times d_{\text{model}}}$ est l'embedding final

## Pourquoi l'Addition?

L'addition permet au modèle de:
1. **Préserver l'information**: Les deux types d'information coexistent
2. **Apprendre à séparer**: Le modèle peut apprendre à distinguer contenu et position
3. **Simplicité**: Plus simple que la concaténation (pas de changement de dimension)

In [ ]:
# ============================================
# COMBINAISON: Token + Positional
# ============================================

print("=" * 60)
print("COMBINAISON: TOKEN + POSITIONAL EMBEDDINGS")
print("=" * 60)

# Configuration
vocab_size = 1000
max_seq_len = 100
d_model = 128
batch_size = 4
seq_len = 10

print(f"\nConfiguration:")
print(f"  - Vocabulaire: {vocab_size}")
print(f"  - Longueur max: {max_seq_len}")
print(f"  - Dimension: {d_model}")
print(f"  - Batch: {batch_size}")
print(f"  - Séquence: {seq_len}")

# Créer les deux couches
print(f"\n1. Création des couches:")
token_emb = TokenEmbedding(vocab_size=vocab_size, d_model=d_model)
pos_emb = PositionalEmbedding(max_seq_len=max_seq_len, d_model=d_model)
print(f"   ✓ TokenEmbedding créé")
print(f"   ✓ PositionalEmbedding créé")

# Créer des token IDs
print(f"\n2. Création de token IDs:")
token_ids = torch.randint(0, vocab_size, (batch_size, seq_len))
print(f"   ✓ Shape: {token_ids.shape}")

# Calculer les embeddings de tokens
print(f"\n3. Calcul des token embeddings:")
token_embeddings = token_emb(token_ids)
print(f"   ✓ Shape: {token_embeddings.shape}")

# Calculer les encodages positionnels
print(f"\n4. Calcul des positional encodings:")
positional_encodings = pos_emb(token_ids)
print(f"   ✓ Shape: {positional_encodings.shape}")

# Combiner par addition
print(f"\n5. Combinaison par addition:")
combined_embeddings = token_embeddings + positional_encodings
print(f"   ✓ Shape: {combined_embeddings.shape}")
print(f"   ✓ Formule: E_input = E_token + E_position")

# Vérification
print(f"\n6. Vérification:")
expected_shape = (batch_size, seq_len, d_model)
print(f"   - Shape attendue: {expected_shape}")
print(f"   - Shape obtenue: {combined_embeddings.shape}")
assert combined_embeddings.shape == expected_shape
print(f"   ✓ SUCCÈS: Les embeddings sont correctement combinés!")

# Statistiques
print(f"\n7. Statistiques des embeddings combinés:")
print(f"   - Moyenne: {combined_embeddings.mean().item():.6f}")
print(f"   - Écart-type: {combined_embeddings.std().item():.6f}")
print(f"   - Min: {combined_embeddings.min().item():.6f}")
print(f"   - Max: {combined_embeddings.max().item():.6f}")

print("\n" + "=" * 60)
print("✓ Combinaison terminée avec succès!")
print("=" * 60)

## Visualisation de l'Espace d'Embedding

Visualisons l'espace d'embedding en réduisant la dimensionnalité avec **PCA** (Principal Component Analysis) et **t-SNE** (t-Distributed Stochastic Neighbor Embedding).

### PCA vs t-SNE

- **PCA**: Projection linéaire qui préserve la variance globale
- **t-SNE**: Projection non-linéaire qui préserve les structures locales

Ces visualisations nous aident à comprendre comment les embeddings sont organisés dans l'espace.

In [ ]:
# Visualisation de l'espace d'embedding avec PCA
print("=" * 60)
print("VISUALISATION: ESPACE D'EMBEDDING (PCA)")
print("=" * 60)

# Récupérer la matrice d'embedding complète
embedding_matrix_torch = token_emb.get_embedding_matrix().detach().numpy()
print(f"\nMatrice d'embedding shape: {embedding_matrix_torch.shape}")

# Sélectionner un sous-ensemble de tokens pour la visualisation
num_tokens_to_plot = 200
embeddings_subset = embedding_matrix_torch[:num_tokens_to_plot]

# Appliquer PCA pour réduire à 2D
print(f"\nApplication de PCA (128D → 2D)...")
pca = PCA(n_components=2)
embeddings_2d_pca = pca.fit_transform(embeddings_subset)
print(f"✓ PCA terminée")
print(f"  - Variance expliquée: {pca.explained_variance_ratio_.sum():.2%}")

# Visualisation PCA
fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(embeddings_2d_pca[:, 0], embeddings_2d_pca[:, 1],
                    c=range(num_tokens_to_plot), cmap='viridis',
                    alpha=0.6, s=50)

ax.set_xlabel('Première Composante Principale', fontsize=12)
ax.set_ylabel('Deuxième Composante Principale', fontsize=12)
ax.set_title(f'Espace d\'Embedding (PCA)\n{num_tokens_to_plot} premiers tokens',
             fontsize=14, fontweight='bold')

# Colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Token ID', rotation=270, labelpad=20)

# Annoter quelques points
for i in [0, 50, 100, 150]:
    if i < num_tokens_to_plot:
        ax.annotate(f'Token {i}', 
                   xy=(embeddings_2d_pca[i, 0], embeddings_2d_pca[i, 1]),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  - Les embeddings sont initialisés aléatoirement")
print("  - Après l'entraînement, les tokens similaires seraient proches")
print("  - PCA capture les directions de plus grande variance")

In [ ]:
# Visualisation avec t-SNE (plus lent mais capture mieux les structures locales)
print("=" * 60)
print("VISUALISATION: ESPACE D'EMBEDDING (t-SNE)")
print("=" * 60)

# Appliquer t-SNE
print(f"\nApplication de t-SNE (128D → 2D)...")
print("⏳ Cela peut prendre quelques secondes...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_2d_tsne = tsne.fit_transform(embeddings_subset)
print(f"✓ t-SNE terminée")

# Visualisation t-SNE
fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(embeddings_2d_tsne[:, 0], embeddings_2d_tsne[:, 1],
                    c=range(num_tokens_to_plot), cmap='plasma',
                    alpha=0.6, s=50)

ax.set_xlabel('Dimension t-SNE 1', fontsize=12)
ax.set_ylabel('Dimension t-SNE 2', fontsize=12)
ax.set_title(f'Espace d\'Embedding (t-SNE)\n{num_tokens_to_plot} premiers tokens',
             fontsize=14, fontweight='bold')

# Colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Token ID', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("  - t-SNE préserve mieux les voisinages locaux que PCA")
print("  - Les clusters apparaîtraient après l'entraînement")
print("  - Utile pour visualiser les relations sémantiques")

---
## Récapitulatif: Shape Checks

Vérifions toutes les dimensions à travers le pipeline complet d'embeddings.

In [ ]:
# ============================================
# RÉCAPITULATIF: SHAPE CHECKS COMPLETS
# ============================================

print("=" * 60)
print("RÉCAPITULATIF: SHAPE CHECKS COMPLETS")
print("=" * 60)

# Configuration
vocab_size = 1000
max_seq_len = 100
d_model = 128
batch_size = 4
seq_len = 10

print(f"\nConfiguration:")
print(f"  vocab_size = {vocab_size}")
print(f"  max_seq_len = {max_seq_len}")
print(f"  d_model = {d_model}")
print(f"  batch_size = {batch_size}")
print(f"  seq_len = {seq_len}")

# Pipeline complet
print(f"\n" + "=" * 60)
print("PIPELINE COMPLET")
print("=" * 60)

# Étape 1: Token IDs
print(f"\n1. Token IDs (input):")
token_ids = torch.randint(0, vocab_size, (batch_size, seq_len))
print(f"   Shape: {token_ids.shape}")
print(f"   ✓ ({batch_size}, {seq_len})")

# Étape 2: Token Embeddings
print(f"\n2. Token Embeddings:")
token_emb = TokenEmbedding(vocab_size, d_model)
token_embeddings = token_emb(token_ids)
print(f"   Shape: {token_embeddings.shape}")
print(f"   ✓ ({batch_size}, {seq_len}, {d_model})")

# Étape 3: Positional Encodings
print(f"\n3. Positional Encodings:")
pos_emb = PositionalEmbedding(max_seq_len, d_model)
positional_encodings = pos_emb(token_ids)
print(f"   Shape: {positional_encodings.shape}")
print(f"   ✓ ({batch_size}, {seq_len}, {d_model})")

# Étape 4: Combined Embeddings
print(f"\n4. Combined Embeddings (Token + Positional):")
combined_embeddings = token_embeddings + positional_encodings
print(f"   Shape: {combined_embeddings.shape}")
print(f"   ✓ ({batch_size}, {seq_len}, {d_model})")

# Vérification finale
print(f"\n" + "=" * 60)
print("VÉRIFICATION FINALE")
print("=" * 60)

expected_shape = (batch_size, seq_len, d_model)
print(f"\nShape attendue: {expected_shape}")
print(f"Shape obtenue: {combined_embeddings.shape}")

if combined_embeddings.shape == expected_shape:
    print(f"\n✅ SUCCÈS: Toutes les dimensions sont correctes!")
    print(f"\nLes embeddings sont prêts à être utilisés par le transformer!")
else:
    print(f"\n❌ ERREUR: Les dimensions ne correspondent pas!")

print("\n" + "=" * 60)

---
# Résumé et Prochaines Étapes

## Ce que nous avons appris

### 1. Token Embeddings
- Convertissent des IDs discrets en vecteurs denses
- Matrice de shape `(vocab_size, d_model)`
- Poids **apprenables** (mis à jour pendant l'entraînement)
- Capturent le **contenu sémantique** des tokens

### 2. Positional Embeddings
- Encodent la position des tokens dans la séquence
- Utilisation de fonctions sinusoïdales: $\sin$ et $\cos$
- Poids **fixes** (calculés une fois, jamais modifiés)
- Permettent au modèle de comprendre l'**ordre des mots**

### 3. Combinaison
- Addition simple: $E_{\text{input}} = E_{\text{token}} + E_{\text{position}}$
- Préserve la dimension: `(batch_size, seq_len, d_model)`
- Les deux types d'information coexistent

### 4. Visualisations
- **Heatmap**: Patterns sinusoïdaux des encodages positionnels
- **PCA**: Structure globale de l'espace d'embedding
- **t-SNE**: Structures locales et voisinages

## Formules Clés

**Token Embedding:**
$$\text{embedding}(i) = E[i] \in \mathbb{R}^{d_{\text{model}}}$$

**Positional Encoding:**
$$PE_{(\text{pos}, 2i)} = \sin\left(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\right)$$
$$PE_{(\text{pos}, 2i+1)} = \cos\left(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\right)$$

**Embedding Final:**
$$E_{\text{input}} = E_{\text{token}} + E_{\text{position}}$$

## Prochaines Étapes

Maintenant que nous avons les embeddings d'entrée, nous pouvons passer au **mécanisme d'attention** (Hour 2):

1. **Scaled Dot-Product Attention**: Comment les tokens s'attendent mutuellement
2. **Causal Masking**: Empêcher le modèle de voir le futur
3. **Multi-Head Attention**: Capturer différentes relations en parallèle

## Exercices Suggérés

1. **Modifier les dimensions**: Essayez différentes valeurs de `d_model` (64, 256, 512)
2. **Visualiser plus de tokens**: Augmentez `num_tokens_to_plot` dans les visualisations
3. **Comparer les initialisations**: Essayez différentes graines aléatoires
4. **Analyser les fréquences**: Tracez les valeurs d'encodage positionnel pour différentes dimensions
5. **Expérimenter avec seq_len**: Observez comment les encodages changent pour des séquences plus longues

---

**Félicitations! Vous avez complété le notebook sur les Embeddings! 🎉**

Passez au notebook suivant: `02_attention/01_scaled_dot_product_attention.ipynb`